In [8]:
import pandas
import re
import gensim
import spacy
import numpy as np
import pyLDAvis.gensim
from nltk.corpus import stopwords
from pprint import pprint
from gensim.test.utils import datapath
from scipy.stats import entropy

In [9]:
remove_lines = ["It's not an attachment -- it's stored online. To open this item, just clickthe link above.",
                "I've shared an item with you:"]


def tokenizing(doc):
    """
    Convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.
    """
    return gensim.utils.simple_preprocess(str(doc), deacc=True, min_len=4, max_len=15)


def preprocessed_doc(doc):
    """
    Preprocessing a single document
    """
    for line in remove_lines:
        doc = doc.replace(line, '')
    
    doc = re.sub(r'https?\S*\s?', '', doc)
    doc = re.sub(r'\s\S*\.ac\.in\S*\s?', ' ', doc)
    doc = re.sub(r'\s\S*\.edu\S*\s?', ' ', doc)
    doc = re.sub(r'\s\S*\.com\S*\s?', ' ', doc)
    doc = re.sub(r'www\.\S*\s?', '', doc)
    doc = re.sub(r'[^a-zA-Z\s]', '', doc)
    doc = re.sub(r'\s+', ' ', doc)    
    doc = tokenizing(doc)
    return doc
    
    
def preprocessed_data(data):
    """
    Preprocessing the entire data (list of documents)
    """
    return [preprocessed_doc(doc) for doc in data]


df = pandas.read_json('data/processed_iitdh_broadcast-3.json')
data = df.content.values.tolist()
tokenized_data = preprocessed_data(data)


In [10]:
stopwords_ = ['dharwad', 'regard', 'iit', 'please', 'student',
              'institute', 'write', 'thank', 'form', 'technology',
              'would', 'year', 'time', 'follow', 'email', 'room', 'fill', 'date',
              'also', 'indian', 'engineering', 'give', 'get', 'day', 'work', 'india',
              'detail', 'mail', 'interested', 'use', 'request', 'guy', 'may',
              'not', 'link', 'like', 'take', 'make', 'still', 'since', 'keep',
              'secretary', 'come', 'one', 'professor', 'today', 'good',
              'find', 'tomorrow', 'first', 'send', 'system', 'start',
              'information', 'member', 'prof', 'part', 'regretted' 'registrar'
              'new', 'venue', 'attend', 'kindly', 'aug', '-PRON-',
              'image', 'provide', 'well', 'visit', 'do', 'inconvenience'
              'assistant_professor', 'want', 'contact', 'go', 'meet',
              'invite', 'name', 'need', 'opportunity', 'attach',
              'everyone', 'google', 'receive', 'conduct', 'great', 'note', 'affair',
              'available', 'number', 'august', 'many', 'college', 'share',
              'help', 'programme', 'walmi_campus', 'belur_industrial',
              'know', 'hold', 'participation', 'march', 'group', 'walmi'
              'rule', 'see', 'learn', 'hello_everyone', 'indore', 'campus'
              'challenge', 'phd', 'present', 'people', 'saturday', 'open', 'hope',
              'april', 'jan', 'online', 'issue', 'require', 'play', 'thing',
              'back', 'faculty', 'dean', 'summer', 'join', 'next', 'inform',
              'base', 'topic', 'hello', 'session', 'idea', 'model', 'participant',
              'post', 'last', 'change', 'round', 'karnataka_india', 'title', 'list',
              'two', 'bring', 'access', 'app', 'include', 'click', 'sincerely', 'learn'
              'view', 'hour', 'close', 'question', 'create', 'third', 'front', 'arpit', 'agrawal', 'btech',
              'high', 'week', 'gentle_reminder', 'september', 'service', 'area', 'morning',
              'timing', 'inter', 'from', 'subject', 'text', 'fwd', 'forwarded', 'message', 'dear', 'f',
              'iitdh', 'iitg', 'deputation', 'three', 'become', 'karnataka', 'select', 'wbw_prasanna',
              'unsubscribe', 'pron', 'hey', 'cse', 'january', 'february', 'march', 'april', 'june', 'july',
              'august', 'september', 'october', 'november', 'december', 'larsen', 'tourbo', 'tourbo_chair',
              'bombay', 'lot', 'sit', 'try', 'ive_invite', 'till', 'every', 'never', 'near', 'welfare', 'techf',
              'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'secy', 'could',
              'student', 'department', 'first', 'second', 'four', 'feel', 'reminder', 'gentle',  'sure',
              'sign', 'photo', 'welcome', 'assistant', 'already', 'sorry', 'small', 'attachment', 'ever']

stop_words = stopwords.words('english')
stop_words.extend(stopwords_)


In [16]:
bigram = gensim.models.Phrases(tokenized_data, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
nlp = spacy.load('en', disable=['parser', 'ner'])


def remove_stopwords(doc):
    """
    Removes stopwords from a document
    """
    return [word for word in doc
            if word not in stop_words]


def make_bigrams(doc):
    """
    Make bigrams of a document
    """
    return bigram_mod[doc]


def lemmatization(doc):
    """
    Lemmatizes a document
    """
    doc = nlp(" ".join(doc))
    return [token.lemma_ for token in doc]


def processed_doc(doc):
    """
    Processing a document
    """
    doc = remove_stopwords(doc)
    doc = make_bigrams(doc)
    doc = lemmatization(doc)
    doc = remove_stopwords(doc)
    return doc


def processed_data(data):
    """
    Return lemmatized data
    """
    return [processed_doc(doc) for doc in data]

lemmatized_data = processed_data(tokenized_data)

In [17]:
num_of_topics = 10
lda_model = gensim.models.LdaModel.load(datapath('iitdh_bow_10_0.01_0.61'))
id2word = gensim.corpora.Dictionary(lemmatized_data)
corpus = corpus = [id2word.doc2bow(text) for text in lemmatized_data]

tfidf = gensim.models.TfidfModel(corpus)
corpus = tfidf[corpus]


In [18]:
def print_topics():
    """
    Prints top 30 words from each topic
    """
    pprint(lda_model.show_topics(num_of_topics, 30))


def get_doc_topic_distribution(index):
    """
    Get the topic distribution for a given document
    """
    doc = corpus[index]
    return lda_model.get_document_topics(doc)


def get_topic_term():
    """
    Get topic-term matrix
    """
    topic_term_matrix = lda_model.get_topics()
    print(len(topic_term_matrix), 'x', len(topic_term_matrix[0]))
    return topic_term_matrix


def get_term_topics(word_id):
    """
    Get the most relevant topics to the given word
    """
    print('Word is: ', id2word[word_id])
    relevant_topics = lda_model.get_term_topics(word_id)
    relevant_topic_ids = [topic_id[0] for topic_id in relevant_topics]
    for topic_id in relevant_topic_ids:
        print(show_topic(topic_id))
    return relevant_topics


def unseen_doc_topic_distribution(new_doc: list):
    """
        Get topic distribution of a unseen document
    """
    new_doc = [id2word.doc2bow(new_doc)]
    topics = lda_model[new_doc]
    return sorted(topics[0][0], key=lambda x: x[1], reverse=True)


def doc_topic_matrix():
    """
    Get document topic matrix
    """
    matrix = np.zeros(shape=(len(corpus), num_of_topics))
    for i in range(len(corpus)):
        topic_dist = get_doc_topic_distribution(i)
        for topic, prob in topic_dist:
            matrix[i][topic] = prob

    return matrix


def jensen_shannon(query, matrix):
    """
    This function implements a Jensen-Shannon similarity
    between the input query (an LDA topic distribution for a document)
    and the entire corpus of topic distributions.
    It returns an array of length M where M is the number of documents in the corpus
    """
    p = query[None, :].T
    q = matrix.T
    m = 0.5 * (p + q)
    # entropy calculated KL divergence
    return np.sqrt(0.5 * (entropy(p, m) + entropy(q, m)))


def get_document(index):
    """
    Index is list or num
    """
    if isinstance(index, list):
        docs = []
        for i in index:
            docs.append(data[i])
        return docs
    return data[index]


def get_similar_docs(new_doc: list):
    """
    This function implements the Jensen-Shannon distance above
    and retruns the top k indices of the smallest jensen shannon distances
    """
    topics = unseen_doc_topic_distribution(new_doc)
    topic_dist = np.zeros(shape=(num_of_topics))
    for topic, prob in topics:
        topic_dist[topic] = prob
    sims = jensen_shannon(topic_dist, doc_topic_matrix())
    docs = list(sims.argsort()[:10])
    return get_document(docs)


def retrieval(doc):
    doc = preprocessed_doc(doc)
    doc = processed_doc(doc)
    return get_similar_docs(doc)


In [22]:
doc = '''
Dear all,
   Prof Krishna Maddaly from Ashoka University, Delhi will be speaking in our seminar.
The talk is for general audience and all are invited. The details of  the talk is as follows:

Title : Random Matrices in Quantum Mechanics
Speaker: Prof Krishna Maddaly
Date & Time: 8-11-2019 (Friday), 4:10 pm.
Venue:  Room No - 023.

Abstract: In this talk we will describe how random matrices play a role in describing some Quantum Mechanical systems and explain some problems connected with such matrices.
'''

In [23]:
similar_docs = retrieval(doc)
print(similar_docs[0])

Subject: Re: A talk by Professor Van Maldeghem: 16 Jan, 2:30 pm, Room 023
Text: A gentle reminder about today's talk.
Amlan

On Mon, Jan 14, 2019 at 12:42 PM Amlan Barua <abarua@iitdh.ac.in> wrote:

> Dear All,
> The talk by Professor Van Maldeghem is scheduled for 16th Jan at 2:30 pm.
> You all are cordially invited in room 023 for the talk. Please find the
> poster attached to know more about the talk.
> Amlan
>



In [24]:
print(similar_docs[1])

Subject: Re: Talk by Dr. S. Vanka at 1400 in Rm no. 119
Text: Dear colleagues and students,

A gentle reminder for the talk today. The venue will be room number 119.

Regards,
Naveen

On Mon, 8 Oct, 2018, 17:39 Naveen Kadayinti, <naveen@iitdh.ac.in> wrote:

>
> Dear Colleagues and Students
>
> We have a visitor, Dr. Sundaram Vanka, to the institute on 11th October.
> He will be giving a talk on 11th October from 1400 to 1500 Hrs. The details
> of the talk are as follows.
>
> Title : Interference Management in Emerging Wireless Networks and Systems
>
> Abstract: An important source of complexity in emerging wireless networks
> is the increasingly diverse set of wireless devices and their applications.
> Interference management is a key challenge in the design and operation of
> such networks, and forms the overall theme that connects the speaker’s work
> in diverse settings. This talk provides a brief overview of some aspects of
> this work and is organized as follows. Part one situates